# Hyperparameter Tuning for ESN on Kepler Time Series

## 1. Problem Statement

In our first experiment with a small Echo State Network (ESN) on the Kepler labelled dataset, the results showed:

- **Accuracy:** 99%
- **AUC:** 0.7
- **Errors:** 5 out of 570  

At first sight, this seems excellent. However, **all errors correspond to stars that actually have exoplanets**, which means the model **never detected any exoplanet**.  

**Problem:** The ESN is biased toward the majority class (no exoplanet) due to the extreme class imbalance in the dataset. The goal of this notebook is to **fine-tune the ESN hyperparameters** to improve detection of exoplanets.

## 2. Initial Setup

We start with the base parameters used previously:

```python
units = 20          # number of neurons in the reservoir
leak_rate = 0.3     # leak rate (memory retention of neurons)
input_scaling = 0.5 # influence of input on the reservoir
spectral_radius = 0.9 # strength of reservoir dynamics
ridge = 1e-6        # regularization
```

## 3. Parameters to Optimize

Based on the tutorial and our observations, we can focus on the following hyperparameters:

Parameter	Description	Range / Notes
units	Number of neurons in the reservoir	20 → 200
leak_rate	Memory of the reservoir	0.1 → 1
input_scaling	Influence of input on reservoir	0.1 → 1
spectral_radius	Strength of dynamics	0.5 → 1.5
ridge	Regularization (ridge)	1e-8 → 1e-2

**Goal:** increase the network's ability to detect the minority class (exoplanets) without destabilizing the reservoir.

## 4. Strategy

1. Fix the random seed to ensure reproducibility.

2. Normalize each series individually (mean 0, std 1).

3. Define a grid or random search over the above hyperparameters.

4. For each configuration:

    - Train the ESN

    - Compute accuracy, AUC and get the number of errors

5. Select hyperparameters that maximize AUC (more informative than accuracy due to imbalance) and minimize the number of errors.

## 5. Load Dataset and Preprocessing

In [2]:
import numpy as np
import pandas as pd

# Load your train/test datasets
train_df = pd.read_csv("../data/exoTrain.csv")
test_df = pd.read_csv("../data/exoTest.csv")

# Extract labels
y_train_np = np.asarray(train_df.iloc[:,0], dtype=int) - 1  # 0 = no exoplanet, 1 = exoplanet
y_test_np  = np.asarray(test_df.iloc[:,0], dtype=int) - 1

# Extract time series data
X_train = train_df.iloc[:, 1:].values[..., np.newaxis]
X_test  = test_df.iloc[:, 1:].values[..., np.newaxis]

# Convert to list of sequences
X_train_seq = [x for x in X_train]
X_test_seq  = [x for x in X_test]

# Normalize per series
X_train_seq = [(x - x.mean()) / (x.std() + 1e-8) for x in X_train_seq]
X_test_seq  = [(x - x.mean()) / (x.std() + 1e-8) for x in X_test_seq]

# Create sequential labels for ESN
y_train_seq = [np.full((x.shape[0], 1), y) for x, y in zip(X_train_seq, y_train_np)]


## 6. Example: Base ESN Training

In [ ]:
from reservoirpy.nodes import Reservoir, Ridge

# Base ESN
reservoir = Reservoir(
    units=20,
    lr=0.3,
    input_scaling=0.5,
    sr=0.9
)

readout = Ridge(ridge=1e-6)

esn = reservoir >> readout

# Train
esn.fit(X_train_seq, y_train_seq)

# Predict
y_pred_seq = esn.run(X_test_seq)
y_pred = np.array([yp.mean() for yp in y_pred_seq])
y_pred_label = (y_pred > 0.5).astype(int)

# Evaluate
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

accuracy = accuracy_score(y_test_np, y_pred_label)
auc = roc_auc_score(y_test_np, y_pred)

Accuracy = 0.9912280701754386,
Area Under the ROC Curve = 0.7139823008849557, Confusion matrice: [[565   0]
 [  5   0]]
Number of errors: 5 on 570


#### Performance analysis

In [7]:
print(f"Accuracy = {round(accuracy*100, 2)}%,\nArea Under the ROC Curve = {round(auc, 2)}")

# Errors indices
errors = np.where(y_test_np != y_pred_label)[0]
print(f"Number of errors: {len(errors)} on {len(y_test_np)}")

Accuracy = 99.12%,
Area Under the ROC Curve = 0.71
Number of errors: 5 on 570



Here are the results obtained in the first experiment with base parameters. They look great, but the network **fails** to recognise stars with an exoplanet.

## 7. Hyperparameter Fine-tuning

Now that the ESN is all set up, we can start the experiment.

### Methodology 

The methodology we propose here is inspired by the paper *[Which Hype for my New Task? Hints and Random Search for Echo State Networks Hyperparameters](https://inria.hal.science/hal-03203318v2/document)*, published in 2021 by Xavier Hinaut and Nathan Trouvain. In this paper, the authors suggest that random search is more efficient than grid search for ESN hyperparameters optimization.

Thus, in the following steps, we will finetune hyperparameters thanks to a random search, instead of a grid search.

### Why random search?

The paper written by Xavier Hinaut and Nathan Trouvain explains that in a grid search, evaluations can be wasted in **low-impact regions**. Indeed, in reservoir computing, overall performance on hyperparameters is rather **non-linear and irregular**. There are a lot of chance than gird-search only sees low-impact regions because of these non-lineariries.

The advantage of random search is that it **samples uniformly** across the full range of each parameter. Hence, it covers more independent combinations and is therefore more likely to hit effective configurations with **fewer trials**. Because only a few hyperparameters strongly influence performance, random search spreads the search more efficiently in order to explore these influential directions. This approach reduces the redundancy inherent in grids and achieves better performance for equivalent search conditions.

### Applying random search to optimise hyperparameters

Here is a script that proposes a simple random search for optimisation, applied to our model and dataset.

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from reservoirpy.nodes import Reservoir

# -----------------------
# Random search parameters
# -----------------------
np.random.seed(99)  # reproducibility
n_configs = 6       # number of random configurations to try

units_range = [5, 10, 15, 20] # Not pushing my computer too hard...
spectral_radius_range = [0.1, 0.9, 1.25, 10.0]
input_scaling_range = [0.1, 0.5, 1.0]
leak_rate_range = [0.1, 0.3, 0.5, 0.9]
rc_connectivity_range = [0.05, 0.1, 0.5]
input_connectivity_range = [0.1, 0.5, 1.0]

# -----------------------
# Run random search
# -----------------------
states = []
params_list = []

# Optionally take only first 500 timesteps to reduce computation
X_subset_seq = [x[:500] for x in X_train_seq]

for i in range(n_configs):
    UNITS = np.random.choice(units_range)
    SPECTRAL_RADIUS = np.random.choice(spectral_radius_range)
    INPUT_SCALING = np.random.choice(input_scaling_range)
    LEAK_RATE = np.random.choice(leak_rate_range)
    RC_CONNECTIVITY = np.random.choice(rc_connectivity_range)
    INPUT_CONNECTIVITY = np.random.choice(input_connectivity_range)
    SEED = np.random.randint(0, 10000)

    reservoir = Reservoir(
        units=UNITS,
        sr=SPECTRAL_RADIUS,
        input_scaling=INPUT_SCALING,
        lr=LEAK_RATE,
        rc_connectivity=RC_CONNECTIVITY,
        input_connectivity=INPUT_CONNECTIVITY,
        seed=SEED
    )

    # Run reservoir on subset
    s = reservoir.run(X_subset_seq)
    states.append(s)

    # Save configuration for reference
    params_list.append({
        "units": UNITS,
        "spectral_radius": SPECTRAL_RADIUS,
        "input_scaling": INPUT_SCALING,
        "leak_rate": LEAK_RATE,
        "rc_connectivity": RC_CONNECTIVITY,
        "input_connectivity": INPUT_CONNECTIVITY,
        "seed": SEED
    })


### Visualisation

#### Spectral Radius

In [ ]:
UNITS_SHOWN = 10

plt.figure(figsize=(15, 4 * n_configs))
for i, s_list in enumerate(states):
    s = np.concatenate(s_list, axis=0) # Concatenate all sequences along the time axis
    plt.subplot(n_configs, 1, i + 1)
    plt.plot(s[:, :UNITS_SHOWN], alpha=0.6)
    plt.ylabel(f"Config {i+1}: sr={params_list[i]['spectral_radius']}, units={params_list[i]['units']}")
plt.xlabel(f"Activations ({UNITS_SHOWN} neurons)")
plt.tight_layout()
plt.show()